# Import Libraries

In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim

# Define the data transforms

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# Load the dataset

In [3]:
train_dataset = datasets.ImageFolder(root='train', transform=transform)
test_dataset = datasets.ImageFolder(root='test', transform=transform)

# Create the data loaders

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define CNN Model

In [5]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 53 * 53)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

# Define the loss function and optimizer

In [6]:

model = CNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Train the Model

In [7]:

for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):

        outputs = model(images)

        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, 10, i + 1, len(train_loader), loss.item()))


Epoch [1/10], Step [1/18], Loss: 0.6677
Epoch [2/10], Step [1/18], Loss: 0.7758
Epoch [3/10], Step [1/18], Loss: 0.5575
Epoch [4/10], Step [1/18], Loss: 0.6335
Epoch [5/10], Step [1/18], Loss: 0.5567
Epoch [6/10], Step [1/18], Loss: 0.3389
Epoch [7/10], Step [1/18], Loss: 0.4074
Epoch [8/10], Step [1/18], Loss: 0.1492
Epoch [9/10], Step [1/18], Loss: 0.1247
Epoch [10/10], Step [1/18], Loss: 0.0444


# Evaluate the model on the test set

In [8]:
correct = 0
total = 0

# Iterate over the test data

In [9]:
for i, (images, labels) in enumerate(test_loader):

    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    correct += (predicted == labels).sum().item()
    total += labels.size(0)

# Calculate the accuracy

In [10]:
accuracy = correct / total * 100

# Print the accuracy

In [11]:
print('Accuracy:', accuracy)

Accuracy: 53.57142857142857
